In [ ]:
# Install packages needed for imports below
# ! pip3 install wavio 
# ! pip3 install librosa
# ! pip3 install sounddevice
# had to install an earlier version of matplotlib in order to get the librosa waveshow function to work
# ! pip3 install matplotlib==3.7.3

In [94]:
# import necessary modules
import numpy as np
from scipy.io import wavfile
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.fft import rfft, rfftfreq
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import numpy as np
from scipy.signal import butter,filtfilt

In [95]:
# Define Sampling Rate in Hz
sr = 44100

# Duration of sound files in seconds
duration = 5

# spectrogram generation function from project 1, used for all questions
def generate_spectrogram(array, sr_in, plot_title, max_freq):
    freq = librosa.amplitude_to_db(np.abs(librosa.stft(array)), ref=np.max)
    fig, ax = plt.subplots()
    plt.ylim(0, max_freq) # limit frequencies plotted to between 0 and 8000 Hz
    plt.title(plot_title) # insert plot title based on function input
    img = librosa.display.specshow(freq, x_axis='time', y_axis='linear',ax=ax, sr=sr_in)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Frequency (Hertz)")
    fig.colorbar(img, ax=ax)

In [ ]:
# Sine Tone Generation

# generate time axis using sample rate and duration from project 1
x = np.linspace(0, duration, sr*duration, endpoint=False)

# apply input frequency of 5kHz to each time sample
frequencies = x*5000

# pass frequencies through sine function
tone1 = np.sin((2*np.pi)*frequencies)

# write sine wave to a wav file
write("team[]-sinetone.wav", sr, tone1)

# generate spectrogram of 5kHz sine wave
generate_spectrogram(tone1, sr, "5kHz Sine Wave", 8000)

In [ ]:
# Chirp Signal Generation

# determine frequency increment per time sample
freq_step = (8000/(sr*duration))/2

# generate time samples
x = np.linspace(0, duration, sr*duration, endpoint=False)

frequencies = x
freq = 0   
   
# iterate through each time array element and multiply by frequency
for i in range(len(frequencies)):
    frequencies[i] = x[i]*freq
    # increment frequency from 0Hz to 8000Hz
    freq = freq+freq_step

tone1 = np.sin((2*np.pi)*frequencies)

write("team[]-chirp.wav", sr, tone1)
generate_spectrogram(tone1, sr, "0Hz to 8kHz Chirp Signal", 8000)

In [ ]:
# Some Fun with Sine Tone 

# Array of frequencies for each of the five notes
note_freq = [466.16, 523.25, 415.3, 207.65, 311.13]
# Array of durations in seconds for each note
seconds = np.array([0.4, 0.586, 0.857, 0.586, 1.571])
# Convert durations in seconds to number of time samples
samples = seconds * (sr*duration/5)

# generate time samples
x = np.linspace(0, duration, sr*duration, endpoint=False)
frequencies = x  
    
# variables used to determine when to switch to next note
lower_limit = 0
upper_limit = samples[0]
current_note = 0

# iterate through all time samples (5sec total)
for i in range(len(frequencies)):
    # if within the duration of a note, multiply by the corresponding frequency
    if lower_limit <= i < upper_limit:
        frequencies[i] = x[i]*note_freq[current_note]
    else:
        # increment to the next note
        current_note = current_note + 1
        # if all notes have been played, multiply the current time sample by zero and 
        # continue iterating through the remaining time samples
        if current_note > len(seconds)-1:
            frequencies[i] = x[i]*0
            continue
        else:
            # set lower limit of the next note to upper limit of the last note
            lower_limit = upper_limit
            # add duration of the next note to the current upper limit 
            upper_limit = upper_limit + samples[current_note]
                
tone1 = np.sin((2*np.pi)*frequencies)

write("team[]-cetk.wav", sr, tone1)
generate_spectrogram(tone1, sr, "Close Encounters Five Tones", 700)

In [ ]:
# Combining Sound Files

# load speech wav file from project 1
wav_speech, sr1 = librosa.load("quick_brown_fox.wav")

# load 5kHz sine wav file
wav_sine, sr2 = librosa.load("team[]-sinetone.wav")

# add the speech and sine arrays together
combined_sound = wav_speech + wav_sine

write("team[]-speechsine.wav", sr1, combined_sound)
generate_spectrogram(combined_sound, sr1, "Speech Signal with 5 kHz Sine Wave", 8000)

In [ ]:
# Speech and Audio Filtering

# load combined sine and speech wav file
speech_sine, sr1 = librosa.load("team[]-speechsine.wav")

# set low pass filter cutoff frequency to 4000Hz
cutoff = 4000     
# calculate nyquist frequency using sampling rate
nyq = 0.5 * sr  
# set filter polynomial order to 5 to completely remove 5kHz sine wave
order = 5      

# normalize cutoff frequency using nyquist frequency
normal_cutoff = cutoff / nyq 

# create a Butterworth lowpass filter using Scipy Library
b, a = butter(order, normal_cutoff, btype='low', analog=False)

# filter the combined wav file using the Butterworth filter
filtered_speech_sine = filtfilt(b, a, speech_sine)

write("team[]-filteredspeechsine.wav", sr1, filtered_speech_sine)
generate_spectrogram(filtered_speech_sine, sr1, "Filtered Speech Signal", 8000)

In [ ]:
# Stereo Fun

# load speech wav file from project 1
wav_speech, sr1 = librosa.load("quick_brown_fox.wav")

# load combined sine and speech file
speech_sine, sr2 = librosa.load("team[]-speechsine.wav")

# combine wav files with speech only on the left channel and speech + sine on the right channel
stereo_fun = np.hstack((wav_speech.reshape(-1, 1), speech_sine.reshape(-1, 1)))

write("team[]-stereospeechsine.wav", sr1, stereo_fun)

generate_spectrogram(wav_speech, sr1, "Stereo Left Channel", 8000)
generate_spectrogram(speech_sine, sr2, "Stereo Right Channel", 8000)